In [1]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import geopandas
import plotly.express as px
import os
from geopandas import GeoDataFrame
from shapely.geometry import Point
import matplotlib.pyplot as plt
import imageio
import mapclassify
import matplotlib
import matplotlib.image as mpimg
import ast
import json
import numpy as np
import shapely.wkt
import math

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib
pd.set_option('display.max_columns', None)
from six.moves import reduce
import scipy
from sklearn.metrics import r2_score
#scipy.stats.pearsonr
import scipy.stats as stats
plt.rcParams.update({'font.family': 'Arial'})
import matplotlib.colors as clr
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.colors import TwoSlopeNorm
from sklearn.cluster import *
from sklearn.decomposition import PCA
from functools import reduce
import scipy.stats

import math
import sklearn
import sklearn.feature_extraction.text
from sklearn import cluster
from sklearn import metrics
import sklearn.manifold #For T-SNE
import sklearn.decomposition #For PCA
import sklearn.pipeline
import sklearn.preprocessing
import sklearn.datasets
import sklearn.decomposition
import sklearn.naive_bayes
import sklearn.tree
import sklearn.ensemble
import sklearn.neural_network
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import classification_report
from collections import Counter
from sklearn.metrics import roc_auc_score
from shapely import wkt
#from mycolorpy import colorlist as mcp
import matplotlib.ticker as mtick
from  matplotlib.colors import LinearSegmentedColormap
from functools import reduce
from scipy import stats

In [2]:
param_2018 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2018/PSO_2018_6params_NYC_norm_28_PSO_15.csv')
table_2018 = pd.read_csv('data/2_data_for_PSO_calibration/table_2018.csv')

param_2019 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2019/PSO_2019_6params_NYC_norm_28_PSO_15.csv')
table_2019 = pd.read_csv('data/2_data_for_PSO_calibration/table_2019.csv')

param_2020 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2020/PSO_2020_6params_NYC_norm_28_PSO_15.csv')
table_2020 = pd.read_csv('data/2_data_for_PSO_calibration/table_2020.csv')

param_2021 = pd.read_csv(
    'data/3_PSO_results/PSO_calirabtion_results/norm_PSO_combination_2021/PSO_2021_6params_NYC_norm_28_PSO_15.csv')
table_2021 = pd.read_csv('data/2_data_for_PSO_calibration/table_2021.csv')

In [3]:
dfs = {'param_2018': param_2018,'param_2019': param_2019,'param_2020': param_2020, 'param_2021': param_2021,
      'table_2018': table_2018,'table_2019': table_2019,'table_2020': table_2020, 'table_2021': table_2021}

In [4]:
vars_definition = {'H_Area_of_store': 'Store area', 
       'R_Percentage_of_Visits_by_brand': 'Chain loyalty', 'J_POI_count_where_store_is': 'POI count',
       'K_POI_diversity_where_store_is': 'POI diversity', 'L_Demographic_similarity': 'Demographic similarity',
       'G_Distance_between_cbg_and_store':'CBG-Store Distance'}

In [5]:
variables = ['H_Area_of_store', 
       'R_Percentage_of_Visits_by_brand', 'J_POI_count_where_store_is',
       'K_POI_diversity_where_store_is', 'L_Demographic_similarity',
       'G_Distance_between_cbg_and_store']

In [6]:
for v in variables:
    param_2018 = param_2018.rename(columns = {v: v + '_2018'})
    param_2019 = param_2019.rename(columns = {v: v + '_2019'})
    param_2020 = param_2020.rename(columns = {v: v + '_2020'})
    param_2021 = param_2021.rename(columns = {v: v + '_2021'})

In [7]:
param_2018 = param_2018.drop(columns = ['cost'])
param_2019 = param_2019.drop(columns = ['cost'])
param_2020 = param_2020.drop(columns = ['cost'])
param_2021 = param_2021.drop(columns = ['cost'])

In [8]:
params = param_2018.merge(param_2019, on = 'cbg').merge(param_2020, on = 'cbg').merge(param_2021, on = 'cbg')
for c in list(params.columns):
    if 'H_Area_of_store' in c:
        params = params[params[c] != 'no visitors']

In [9]:
params['cbg'].nunique()

5502

# KS Test

In [10]:
scipy.stats.kstest(params[variables[0]+'_2018'], params[variables[0]+'_2019'])#[1]

KstestResult(statistic=0.010723373318793167, pvalue=0.9098207114998735, statistic_location=1.834898791686548, statistic_sign=1)

In [11]:
stats.kstest(params[variables[0]+'_2018'], params[variables[0]+'_2019']).pvalue

0.9098207114998735

In [12]:
params.shape

(5502, 25)

In [13]:
def process_pvalue(p_here):
    #same strategy as in R
    if p_here <0.001:
        return '\\text{' + '{:0.3e}'.format(p_here)+'}^\\text{***}'#+'***'
    elif p_here >= 0.001 and p_here < 0.01:
        return '\\text{' + '{:0.3e}'.format(p_here)+'}^\\text{**}'#+'**'
    elif p_here >= 0.01 and p_here < 0.05:
        return '\\text{' + '{:0.3e}'.format(p_here)+'}^\\text{*}'#+'*'
    return str(round(p_here,3))

In [14]:
ks_tests = {'variable': [], '2018~2019': [], '2019~2020': [], '2019~2021': [], '2020~2021': []}
for v in variables:
    ks_tests['variable'].append(vars_definition[v])
    ks_tests['2018~2019'].append(stats.kstest(params[v+'_2018'], params[v+'_2019']).pvalue)
    ks_tests['2019~2020'].append(stats.kstest(params[v+'_2019'], params[v+'_2020']).pvalue)
    ks_tests['2019~2021'].append(stats.kstest(params[v+'_2019'], params[v+'_2021']).pvalue)
    ks_tests['2020~2021'].append(stats.kstest(params[v+'_2020'], params[v+'_2021']).pvalue)
    
ks_tests_df = pd.DataFrame(ks_tests)
ks_tests_df

,variable,2018~2019,2019~2020,2019~2021,2020~2021
0,Store area,0.909821,9.406548e-295,7.222452e-293,9.834698e-03
1,Chain loyalty,0.850672,5.293906e-200,2.989704e-207,3.120209e-01
2,POI count,0.439173,5.150284e-37,5.652137e-87,3.965251e-13
3,POI diversity,0.574231,4.310540e-19,1.239687e-30,2.140085e-02
4,Demographic similarity,0.733842,2.020247e-02,6.277372e-02,9.462703e-01
5,CBG-Store Distance,0.621999,5.138254e-160,0.000000e+00,4.195045e-49


In [15]:
for indx, cont in ks_tests_df.iterrows():
    values = [cont['2018~2019'], cont['2019~2020'], cont['2019~2021'], cont['2020~2021']]
    values = ['$' + process_pvalue(i) + '$' for i in values]
    print(' & '.join([cont['variable']] + values))
    print('\n')

Store area & $0.91$ & $\text{9.407e-295}^\text{***}$ & $\text{7.222e-293}^\text{***}$ & $\text{9.835e-03}^\text{**}$


Chain loyalty & $0.851$ & $\text{5.294e-200}^\text{***}$ & $\text{2.990e-207}^\text{***}$ & $0.312$


POI count & $0.439$ & $\text{5.150e-37}^\text{***}$ & $\text{5.652e-87}^\text{***}$ & $\text{3.965e-13}^\text{***}$


POI diversity & $0.574$ & $\text{4.311e-19}^\text{***}$ & $\text{1.240e-30}^\text{***}$ & $\text{2.140e-02}^\text{*}$


Demographic similarity & $0.734$ & $\text{2.020e-02}^\text{*}$ & $0.063$ & $0.946$


CBG-Store Distance & $0.622$ & $\text{5.138e-160}^\text{***}$ & $\text{0.000e+00}^\text{***}$ & $\text{4.195e-49}^\text{***}$


